In [238]:
import numpy as np

In [239]:
def calc_r(m0, j=999):
    
    if j != 999:
        r = (m0[j+1] - m0[j])/(m0[j+1] + m0[j])
    else:
        r = []
        for i in range(1, len(m0)):

            num = m0[i] - m0[i-1]
            den = m0[i] + m0[i-1]
            r_temp = num/den
            r.append(r_temp)
        r = np.array(r)
    
    return r

In [240]:
def calc_G(m0, r0, r, step=.01, objective = 'Z'):
    
    G = np.zeros((len(m0), len(r0))) # 4x3 
    for j, row in enumerate(G): # for each row
        for i, col in enumerate(G[j]): # for each column
            if (i == j): # changes m_first
                m_first = m0[j]
                m_second = m0[j+1]
                dZ = m_first*step
                r_old = r0[i]
                r_new = (m_second - (m_first+dZ))/(m_second + (m_first+dZ))
                dr = r_old - r_new
                cell_value = dr/dZ
            elif (i==(j-1)): # changes m_second
                m_first = m0[j-1]
                m_second = m0[j]
                dZ = m_second*step
                r_old = r0[i]
                r_new = ((m_second+dZ) - m_first)/((m_second+dZ) + m_first)
                dr = r_old - r_new
                cell_value = dr/dZ
            else:
                cell_value = 0
            G[j][i] = cell_value
            
    if objective == 'Z':
        G = G.T
    else:
        pass
    
    return G

In [241]:
def L1_error(y_true, y_pred):
    n = len(y_true)
    # Calculate the absolute percentage error for each data point
    abs_percentage_error = np.abs((y_pred - y_true) / y_true)
    
    # Calculate the mean absolute percentage error as a percentage of n
    L1 = (np.sum(abs_percentage_error) / n) * 100
    #print(f"L1 error: {L1:.2f}%")
    return L1

def L2_error(y_true, y_pred):
    n = len(y_true)
    # Calculate the squared percentage error for each data point
    squared_percentage_error = ((y_true - y_pred) / y_true) ** 2
    
    # Calculate the mean squared percentage error and take the square root as a percentage of n
    L2 = (np.sqrt(np.sum(squared_percentage_error)) / n) * 100
    #print(f"L2 error: {L2:.2f}%")
    return L2

In [282]:
def MBI(m0, r0, r, step=.01, max_iter=20):
    
    print('This Model-Based Inversion was made with the following parameters')
    print(f'Step change: {step}')
    print(f'max_iter: {max_iter}')
    r_init = r0
    m_init = m0
    
    for i in range(max_iter):
        
        print('='*50, f'Iteration: {i}', '='*50)
        G = calc_G(m0, r0, r, step=.01)
        G_square = np.matmul(G.T, G)
        G_inverse = np.linalg.pinv(G_square)
        G_matmul = np.matmul(G_inverse, G.T)
        error = (r - r0)
        delta_m = np.matmul(G_matmul, error)
        m0 = m0 - delta_m
        r0 = calc_r(m0)
        L1 = L1_error(r, r0)
        L2 = L2_error(r, r0)
        print(f'L1 error: {L1}%')
        print(f'L2 error: {L2}%')
        early_stop_error = .00001
        if L1<early_stop_error and L2<early_stop_error:
            break
    
    print('='*50, 'Results of the MBI', '='*50)
    print(f'Total iterations: {i+1}')
    print(f'L1 error: {L1}%')
    print(f'L2 error: {L2}%')
    print(f'Initial reflectivity model (r): {np.squeeze(r_init)}')
    print(f'Final reflectivity model (r): {np.squeeze(r0)}')
    print(f'Target reflectivity model (r): {np.squeeze(r)}')
    print(f'Initial impedance model (Z): {np.squeeze(m_init)}')
    print(f'Final impedance model (Z): {np.squeeze(m0)}')
    
    return m0, r0
    

# Inversión sísmica basada en modelo

In [283]:
#### Initial data
# observed data
m0 = np.array([1.3, 1.4, 1.2, 1.1]).reshape(4,1) # model [MPa]
r0 = calc_r(m0).reshape(3,1) # synthetic reflectivity profile [seismogram] - Datos estimados
# real data
r = np.array([0.01, -0.012, -0.011]).reshape(3,1) # real reflectivity profile (from seismic data) [seismogram] - Datos observados

In [284]:
print(m0.shape, r0.shape, r.shape)

(4, 1) (3, 1) (3, 1)


In [285]:
m_predicted, r_predicted = MBI(m0, r0, r, step=.01, max_iter=20)

This Model-Based Inversion was made with the following parameters
Step change: 0.01
max_iter: 20
================================================== Iteration: 0 ==================================================
L1 error: 17.725152387715717%
L2 error: 10.326211478736774%
================================================== Iteration: 1 ==================================================
L1 error: 0.08921741454051456%
L2 error: 0.05175962205215356%
================================================== Iteration: 2 ==================================================
L1 error: 0.0004457015417104856%
L2 error: 0.0002585798353962027%
================================================== Iteration: 3 ==================================================
L1 error: 2.2266493298092914e-06%
L2 error: 1.2918354909052335e-06%
================================================== Results of the MBI ==================================================
Total iterations: 4
L1 error: 2.2266493298092914e-06%
L2 error: 1.

In [286]:
r_temp = calc_r([1.26121573, 1.28669472, 1.25618034, 1.22884492])
r_temp

array([ 0.00999996, -0.01199995, -0.01100006])